# Approach

* Firstly a convolutional neural network is used to segment the image, using the bounding boxes directly as a mask. 
* Secondly connected components is used to separate multiple areas of predicted pneumonia.
* Finally a bounding box is simply drawn around every connected component.

# Network

* The network consists of a number of residual blocks with convolutions and downsampling blocks with max pooling.
* At the end of the network a single upsampling layer converts the output to the same shape as the input.

As the input to the network is 256 by 256 (instead of the original 1024 by 1024) and the network downsamples a number of times without any meaningful upsampling (the final upsampling is just to match in 256 by 256 mask) the final prediction is very crude. If the network downsamples 4 times the final bounding boxes can only change with at least 16 pixels.

In [1]:
import os
import csv
import random
import pydicom
import numpy as np
import pandas as pd
import pickle
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load pneumonia locations

Table contains [filename : pneumonia location] pairs per row. 
* If a filename contains multiple pneumonia, the table contains multiple rows with the same filename but different pneumonia locations. 
* If a filename contains no pneumonia it contains a single row with an empty pneumonia location.

The code below loads the table and transforms it into a dictionary. 
* The dictionary uses the filename as key and a list of pneumonia locations in that filename as value. 
* If a filename is not present in the dictionary it means that it contains no pneumonia.

In [17]:
def load_pneumonia_locations(path='./input/stage_2_train_labels.csv'):
    """
    loads all pneumonia bounding boxes from csv into dict.
    :param path: the path to the train labels csv
    :return: pneumonia_locations: a dict of filename: list of bounding boxes
    """
    pneumonia_locations = {}
    with open(os.path.join(path), mode='r') as labels:
        reader = csv.reader(labels)
        # skip header
        next(reader, None)
        for rows in reader:
            filename = rows[0]
            location = rows[1:5]
            pneumonia = rows[5]
            if pneumonia == '1':
                location = [int(float(i)) for i in location]
                if filename in pneumonia_locations:
                    pneumonia_locations[filename].append(location)
                else:
                    pneumonia_locations[filename] = [location]
    return pneumonia_locations


pneumonia_locations = load_pneumonia_locations()


['test_model.py', '.DS_Store', 'models.py', 'requirements.txt', 'train_model.py', 'input', 'kernel (1).ipynb', 'cnn', 'venv', 'separate_train_data.py', '.idea']


# Load filenames

In [18]:
def load_file_names(path='./input/stage_2_train_images', train_ratio=0.8):
    filenames = os.listdir(path)
    random.shuffle(filenames)
    
    train_files_count = int(train_ratio*len(filenames))
    train_filenames = filenames[:train_files_count]
    validation_filenames = filenames[train_files_count:]
    print('Loading {} train files'.format(len(train_filenames)))
    print('Loading {} validation files'.format(len(validation_filenames)))
    return train_filenames, validation_filenames


train_filenames, validation_filenames = load_file_names()

Loading 20547 train files
Loading 5137 validation files


 # Generator
    

In [19]:
class Generator(keras.utils.Sequence):
    
    def __init__(self, folder, filenames, pneumonia_locations=None, batch_size=32, image_size=256, shuffle=True, predict=False):
        """
        :str folder: folder the training/test images are in
        :list(str) filenames: used images filenames 
        :dict(filename: list((x_coordinate, y_coordinate, width, height)) pneumonia_locations: bounding boxes
        :int batch_size: 
        :int image_size: 
        :bool shuffle: whether images should be shuffled, do this for training
        :bool predict: False if training, True if predicting
        """
        self.folder = folder
        self.filenames = filenames
        self.pneumonia_locations = pneumonia_locations
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.predict = predict
        self.on_epoch_end()
        
    def __load__(self, filename):
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array

        # create mask with 1's at pneumonia bounding box
        msk = np.zeros(img.shape)
        filename = filename.split('.')[0]
        if filename in self.pneumonia_locations:
            for location in self.pneumonia_locations[filename]:
                x, y, w, h = location
                msk[y:y+h, x:x+w] = 1
                
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        msk = resize(msk, (self.image_size, self.image_size), mode='reflect') > 0.5
        
        img = np.expand_dims(img, -1)
        msk = np.expand_dims(msk, -1)
        return img, msk
    
    def __loadpredict__(self, filename):
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        img = np.expand_dims(img, -1)
        return img
        
    def __getitem__(self, index):
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        
        if self.predict:
            imgs = [self.__loadpredict__(filename) for filename in filenames]
            imgs = np.array(imgs)
            return imgs, filenames
        
        else:
            items = [self.__load__(filename) for filename in filenames]
            imgs, msks = zip(*items)
            imgs = np.array(imgs)
            msks = np.array(msks)
            return imgs, msks
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.filenames)
        
    def __len__(self):
        if self.predict:
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            return int(len(self.filenames) / self.batch_size)

# Network

In [20]:
def downsample(n_outputs, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(n_outputs, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x


def resblock(n_outputs, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(n_outputs, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(n_outputs, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])


def create_model(input_size, n_outputs, n_blocks=4, depth=3):
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(n_outputs, 3, padding='same', use_bias=False)(inputs)
    for d in range(depth):
        n_outputs = n_outputs * 2
        x = downsample(n_outputs, x)
        for b in range(n_blocks):
            x = resblock(n_outputs, x)

    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Train network


In [25]:
def jaccard_loss(y_true, y_pred, smoothing=1e-12):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    sum_ = tf.reduce_sum(y_true + y_pred)
    score = (smoothing + intersection ) / (sum_ - intersection + smoothing)
    return 1 - score


def jaccard_bce_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * jaccard_loss(y_true, y_pred)


def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))


# create network and compiler
model = create_model(input_size=256, n_outputs=32, n_blocks=4, depth=3)
model.compile(optimizer='adam',
              loss=jaccard_bce_loss,
              metrics=['accuracy', mean_iou])

try:
    model.load_weights('pneumonia_cnn.h5')
except OSError:
    print("Could not load model, starting from random.")

def cosine_annealing(x):
    base_learning_rate = 0.001
    epochs = 25
    return base_learning_rate*(np.cos(np.pi*x/epochs)+1.)/2


learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

folder = '../input/stage_2_train_images'
train_gen = Generator(folder, train_filenames, pneumonia_locations, batch_size=32, image_size=256, shuffle=True, predict=False)
valid_gen = Generator(folder, validation_filenames, pneumonia_locations, batch_size=32, image_size=256, shuffle=False, predict=False)

history = model.fit_generator(
    train_gen, 
    validation_data=valid_gen, 
    callbacks=[learning_rate], 
    epochs=1, 
    workers=4, 
    use_multiprocessing=True
)

model.save_weights('pneumonia_cnn.h5')
print("model saved.")

ValueError: When passing a list as loss, it should have one entry per model outputs. The model has 1 outputs, but you passed loss=[<function jaccard_loss at 0x123f1ac80>, <function binary_crossentropy at 0x118d95e60>]

In [ ]:
def add_bounding_box(axidx, axarr, msk, bounding_box_threshold=0.5, edgecolor='b'):
    comp = msk[:, :, 0] > bounding_box_threshold
    comp = measure.label(comp)
    for region in measure.regionprops(comp):
        y, x, y2, x2 = region.bbox
        height = y2 - y
        width = x2 - x
        axarr[axidx].add_patch(patches.Rectangle((x, y), width, height, linewidth=2, edgecolor=edgecolor, facecolor='none'))


def validate_model(model, validation_generator, bounding_box_threshold=0.5):
    for imgs, msks in validation_generator:
        preds = model.predict(imgs)
        f, axarr = plt.subplots(4, 8, figsize=(20,15))
        axarr = axarr.ravel()
        axidx = 0
        
        for img, msk, pred in zip(imgs, msks, preds):
            axarr[axidx].imshow(img[:, :, 0])
            add_bounding_box(axidx, axarr, msk, bounding_box_threshold)
            add_bounding_box(axidx, axarr, pred, bounding_box_threshold, edgecolor='r')
            axidx += 1
        plt.show()
        break
        
        
validate_model(model, valid_gen)


# Predict test images

In [ ]:
folder = '../input/stage_2_test_images'
test_filenames = os.listdir(folder)
print('{} test samples:'.format(len(test_filenames)))

test_gen = Generator(folder, test_filenames, None, batch_size=25, image_size=256, shuffle=False, predict=True)

submission_dict = {}
for imgs, filenames in test_gen:
    preds = model.predict(imgs)
    for pred, filename in zip(preds, filenames):
        pred = resize(pred, (1024, 1024), mode='reflect')
        comp = pred[:, :, 0] > 0.5
        comp = measure.label(comp)
        predictionString = ''
        for region in measure.regionprops(comp):
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x
            confidence = np.mean(pred[y:y+height, x:x+width])
            predictionString += '{confidence} {x} {y} {width} {height} '.format(confidence=confidence, x=x, y=y, width=width, height=height)

        filename = filename.split('.')[0]
        submission_dict[filename] = predictionString
    if len(submission_dict) >= len(test_filenames):
        break


sub = pd.DataFrame.from_dict(submission_dict, orient='index')
sub.index.names = ['patientId']
sub.columns = ['PredictionString']
sub.to_csv('submission.csv')